In [1]:
import pandas as pd

EXCEL_FILE = "data/Muesli Project raw data 21-3.xlsx"

def import_xls(file, sheet, header=0):
    df = pd.read_excel(file, sheet_name = sheet, header=header)
    return df

    

In [2]:
df_orders = import_xls(EXCEL_FILE, "Orders", header=1)
df_order_process = import_xls(EXCEL_FILE, "Order Process Data")
df_intern = import_xls(EXCEL_FILE, "InternData Study")
df_campaign = import_xls(EXCEL_FILE, "Campaign Data")
df_list = [df_orders, df_order_process, df_intern, df_campaign]


In [3]:
def column_rename(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [4]:
for df in df_list:
    column_rename(df)
    #display(df)


In [5]:
# for df in df_list:
#     display(df.info())
#     display(df.describe())

In [52]:
# 
num_unique_orders = df_orders['order_id'].nunique()
num_unique_order_process = df_order_process['order_id'].nunique()
num_unique_dif = num_unique_orders - num_unique_order_process
print("We don't have all the order ids in the order process dataset. \nContrary to company assumptions there is a  difference", num_unique_dif)

# Nothing found 
# ~ = not in


df_order_process[~df_order_process['order_id'].isin(df_orders['order_id'])].count()


We don't have all the order ids in the order process dataset. 
Contrary to company assumptions there is a  difference 2007


order_id              0
order_date            0
on_truck_scan_date    0
processing_mode       0
dtype: int64

In [6]:
df_list = [df_orders, df_order_process, df_intern, df_campaign]

df_all = pd.merge(df_orders, df_order_process, how='left', on='order_id')
df_all = pd.merge(df_all, df_intern, how='left', on='order_id')
df_all = pd.merge(df_all, df_campaign, how='left', on='order_id')

df = pd.merge(df_orders, df_order_process, how='inner', on='order_id') 

In [53]:
# Doublecheck if we have the same amount of unique order ids in merged df as in the df_order_process
display(df['order_id'].nunique())

display(df_all.head())
display(df.head())

display(df_all.info())
display(df.info()) 

3002

,order_id,order_date_x,customer_id,customer_name_x,origin_channel,country/region,city,state,postal_code,region,...,sub-category,product_id,quantity,order_date_y,on_truck_scan_date,processing_mode,ready_to_ship_date,pickup_date,arrival_scan_date,customer_name_y
0,CA-2017-103800,2017-01-03,DP-13000,Darren Powers,Email,United States,Houston,Texas,77095.0,Central,...,Nuts and more,OFF-PA-10000174,2.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
1,CA-2017-112326,2017-01-04,PO-19195,Phillina Ober,Facebook,United States,Naperville,Illinois,60540.0,Central,...,No Taste All Power,OFF-LA-10003223,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
2,CA-2017-112326,2017-01-04,PO-19195,Phillina Ober,Facebook,United States,Naperville,Illinois,60540.0,Central,...,Steel Cut,OFF-ST-10002743,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
3,CA-2017-112326,2017-01-04,PO-19195,Phillina Ober,Facebook,United States,Naperville,Illinois,60540.0,Central,...,Super Fibre Boost,OFF-BI-10004094,2.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
4,CA-2017-141817,2017-01-05,MB-18085,Mick Brown,Email,United States,Philadelphia,Pennsylvania,19143.0,East,...,Super Mega Protein,OFF-AR-10003478,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN


,order_id,order_date,customer_id,origin_channel,category,sub-category,product_id,quantity,on_truck_scan_date,processing_mode,pretransportation_duration,weekday
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-09,Standard Processing,7,2
1,CA-2019-160304,2019-01-02,BM-11575,Sales,Toasted Muesli,With Fruit,FUR-BO-10004709,3.0,2019-01-09,Standard Processing,7,2
2,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-09,Standard Processing,6,3
3,CA-2019-105207,2019-01-03,BO-11350,Sales,Toasted Muesli,Sweetened,FUR-TA-10000617,7.0,2019-01-09,Standard Processing,6,3
4,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-07,Express,4,3


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10331 entries, 0 to 10330
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            10331 non-null  object        
 1   order_date_x        10331 non-null  datetime64[ns]
 2   customer_id         10331 non-null  object        
 3   customer_name_x     10331 non-null  object        
 4   origin_channel      10331 non-null  object        
 5   country/region      10331 non-null  object        
 6   city                10331 non-null  object        
 7   state               10331 non-null  object        
 8   postal_code         10320 non-null  float64       
 9   region              10331 non-null  object        
 10  category            10331 non-null  object        
 11  sub-category        10331 non-null  object        
 12  product_id          10331 non-null  object        
 13  quantity            10331 non-null  float64   

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5901 entries, 0 to 5900
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_id                    5901 non-null   object        
 1   order_date                  5901 non-null   datetime64[ns]
 2   customer_id                 5901 non-null   object        
 3   origin_channel              5901 non-null   object        
 4   category                    5901 non-null   object        
 5   sub-category                5901 non-null   object        
 6   product_id                  5901 non-null   object        
 7   quantity                    5901 non-null   float64       
 8   on_truck_scan_date          5901 non-null   datetime64[ns]
 9   processing_mode             5901 non-null   object        
 10  pretransportation_duration  5901 non-null   int64         
 11  weekday                     5901 non-null   int64       

None

In [8]:
df["order_id"].nunique()
drop_list = ['customer_name', 'country/region', 'city', 'state', 'postal_code', 'region'] 
df = df.drop(drop_list, axis=1)


In [9]:
display(df.head(10))


,order_id,order_date_x,customer_id,origin_channel,category,sub-category,product_id,quantity,order_date_y,on_truck_scan_date,processing_mode
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-02,2019-01-09,Standard Processing
1,CA-2019-160304,2019-01-02,BM-11575,Sales,Toasted Muesli,With Fruit,FUR-BO-10004709,3.0,2019-01-02,2019-01-09,Standard Processing
2,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-03,2019-01-09,Standard Processing
3,CA-2019-105207,2019-01-03,BO-11350,Sales,Toasted Muesli,Sweetened,FUR-TA-10000617,7.0,2019-01-03,2019-01-09,Standard Processing
4,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-03,2019-01-07,Express
5,US-2019-116365,2019-01-03,CA-12310,Sales,Special Projects Muesil,Gluten Free,TEC-AC-10002217,2.0,2019-01-03,2019-01-09,Standard Processing
6,US-2019-116365,2019-01-03,CA-12310,Sales,Special Projects Muesil,Gluten Free,TEC-AC-10002942,3.0,2019-01-03,2019-01-09,Standard Processing
7,US-2019-116365,2019-01-03,CA-12310,Sales,Special Projects Muesil,Only Oats,TEC-PH-10002890,5.0,2019-01-03,2019-01-09,Standard Processing
8,CA-2019-158211,2019-01-04,BP-11185,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10002026,6.0,2019-01-04,2019-01-09,Standard Processing
9,CA-2019-158211,2019-01-04,BP-11185,Sales,Power Muesli,Super Mega Protein,OFF-AR-10004078,1.0,2019-01-04,2019-01-09,Standard Processing


In [10]:
df[df["order_date_x"] != df["order_date_y"]]
df[df["order_date_x"] == df["order_date_y"]].count()



order_id              5901
order_date_x          5901
customer_id           5901
origin_channel        5901
category              5901
sub-category          5901
product_id            5901
quantity              5901
order_date_y          5901
on_truck_scan_date    5901
processing_mode       5901
dtype: int64

In [11]:
df.drop("order_date_y", axis=1, inplace =True)


In [12]:
df.rename(columns = {"order_date_x": "order_date"}, inplace=True)
display(df)

,order_id,order_date,customer_id,origin_channel,category,sub-category,product_id,quantity,on_truck_scan_date,processing_mode
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-09,Standard Processing
1,CA-2019-160304,2019-01-02,BM-11575,Sales,Toasted Muesli,With Fruit,FUR-BO-10004709,3.0,2019-01-09,Standard Processing
2,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-09,Standard Processing
3,CA-2019-105207,2019-01-03,BO-11350,Sales,Toasted Muesli,Sweetened,FUR-TA-10000617,7.0,2019-01-09,Standard Processing
4,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-07,Express
...,...,...,...,...,...,...,...,...,...,...
5896,CA-2020-126221,2020-12-30,CC-12430,Facebook,Power Muesli,Mega Protein,OFF-AP-10002457,2.0,2021-01-06,Standard Processing
5897,CA-2020-143259,2020-12-30,PO-18865,Email,Special Projects Muesil,Only Oats,TEC-PH-10004774,7.0,2021-01-06,Standard Processing
5898,CA-2020-143259,2020-12-30,PO-18865,Email,Power Muesli,Super Fibre Boost,OFF-BI-10003684,3.0,2021-01-06,Standard Processing
5899,CA-2020-143259,2020-12-30,PO-18865,Email,Toasted Muesli,With Fruit,FUR-BO-10003441,4.0,2021-01-06,Standard Processing


In [13]:
df["pretransportation_duration"] = df["on_truck_scan_date"] - df["order_date"]
df["pretransportation_duration"] = df["pretransportation_duration"].dt.days


In [14]:
df.describe()

,quantity,pretransportation_duration
count,5901.000000,5901.000000
mean,3.782071,6.119641
std,2.212742,2.509086
min,1.000000,0.000000
25%,2.000000,5.000000
50%,3.000000,7.000000
75%,5.000000,8.000000
max,14.000000,12.000000


In [16]:
df['weekday'] = df['order_date'].dt.weekday

0       2
1       2
2       3
3       3
4       3
       ..
5896    2
5897    2
5898    2
5899    2
5900    2
Name: weekday, Length: 5901, dtype: int64

In [18]:
df.head()

,order_id,order_date,customer_id,origin_channel,category,sub-category,product_id,quantity,on_truck_scan_date,processing_mode,pretransportation_duration,weekday
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-09,Standard Processing,7,2
1,CA-2019-160304,2019-01-02,BM-11575,Sales,Toasted Muesli,With Fruit,FUR-BO-10004709,3.0,2019-01-09,Standard Processing,7,2
2,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-09,Standard Processing,6,3
3,CA-2019-105207,2019-01-03,BO-11350,Sales,Toasted Muesli,Sweetened,FUR-TA-10000617,7.0,2019-01-09,Standard Processing,6,3
4,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-07,Express,4,3


In [59]:
# Create two labels of delivery date depending on express/ non/express processing(see flowchart)
same_arrival_day = [0, 2, 5, 6]
different_arrival_day = [1, 3, 4]

# Devide the dataframe into two groups with regard to these labels
same_day = df[df['weekday'].isin(same_arrival_day)]
different_days = df[df['weekday'].isin(different_arrival_day)]

# Doublecheck if the division is done correctly
check_difference = df['order_id'].count() - same_day['order_id'].count() - different_days['order_id'].count()
display(f'The Difference should be zero: ', check_difference)

'The Difference should be zero: '

0

In [60]:
# Compare descriptive statistics of each dataset
display('Same Day' ,same_day.describe())
display('Different Day' ,different_days.describe())

# Comparison:
# quantities: shows very equal distribution, though same_day orders are from 4 days and different days are from 3
 # Tuesday, Thursday and Friday are days with a lot of incoming orders


'Same Day'

,quantity,pretransportation_duration,weekday
count,2932.000000,2932.000000,2932.000000
mean,3.760232,6.438608,2.479877
std,2.180717,2.638581,2.308353
min,1.000000,0.000000,0.000000
25%,2.000000,5.000000,0.000000
50%,3.000000,7.000000,2.000000
75%,5.000000,9.000000,5.000000
max,14.000000,12.000000,6.000000


'Different Day'

,quantity,pretransportation_duration,weekday
count,2969.000000,2969.000000,2969.000000
mean,3.803638,5.804648,2.604244
std,2.244079,2.332292,1.237605
min,1.000000,0.000000,1.000000
25%,2.000000,5.000000,1.000000
50%,3.000000,6.000000,3.000000
75%,5.000000,8.000000,4.000000
max,14.000000,11.000000,4.000000


In [ ]:
# We should analyse and visualize the days with the most incoming orders
# e.g. Barchart of orderquantities and weekdays


# Combine pretransportation_duration with standard_processing-column